In [ ]:
import numpy as np
import os
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage import io, color, transform
import scipy
from scipy.ndimage.filters import convolve
from scipy.ndimage.filters import gaussian_laplace, generic_filter, gaussian_filter
from numpy.random import rand, randint
from matplotlib.patches import Circle

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [ ]:
Scale = 2
k = 1.3
Level = 13
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()

In [ ]:
def sup_fnc(buffer):
    buffer_length = buffer.shape[0]
    center = int(buffer_length / 2)
    return buffer[center] if buffer[center] == buffer.max() else 0

In [ ]:
def down_func(img, Scale, Level, k):

    height = img.shape[0]     
    width= img.shape[1]
    
    ret = np.empty((height, width, Level))
    sampled_image = img
    sigma = Scale
    for i in range(Level):
        X = gaussian_laplace(sampled_image, sigma = sigma) ** 2 
        ret[:, :, i] = transform.resize(X, (height, width), order = 2)
        sampled_image = transform.rescale(sampled_image, 1.0 / k, order = 2)
#     print(ret)
    return ret

In [ ]:
def Print_down():
    ret = down_func(img, Scale,Level,k)
    ret = scipy.ndimage.filters.generic_filter(ret, sup_fnc, size = (3, 3, 3))
    ret = ret/np.max(ret)
    #cx, cy, rad = get_blobs(ret, 0.075)
    #show_all_circles(img, cx, cy, rad, color='r')
        
    cx, cy, count = np.where(ret > 0.15)
    v=(cx.shape[0])
    rad = np.empty(v)
    count_len=len(count)
    for i in range(count_len):
        sigma = Scale * k ** count[i]
        rad[i] = np.sqrt(2) * sigma
    show_all_circles(img, cy, cx, rad, color='r')
    return

In [ ]:
IMG_DIR = 'blobs-data'
for filename in os.listdir(IMG_DIR):
    img = mpimg.imread(filename)
    img = rgb2gray(img)
    plt.imshow(img, cmap='gray')
    plt.show()
    start = time.time()
    Print_down()
    print(time.time()-start)